# Indicadores de Trazabilidad 



## Importación de datos, librerias

In [1]:
# Importa librerias necesarias para el análisis

import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
# Importa datos de población regional y crea DF


df_poblacion = pd.read_csv('data/aysen_population.csv') #Archivo ubicado dentro de la carpeta data

df_poblacion.head(2)

,Unnamed: 0,orden,nombre_region,codigo_region,nombre_provincia,codigo_provincia,nombre_comuna,codigo_comuna,edad_hombres,edad_mujeres,total
0,0,7172,Región de Aysén del General Carlos Ibáñez del ...,11,Coyhaique,111,Coyhaique,11101,28763,29055,57818
1,1,7194,Región de Aysén del General Carlos Ibáñez del ...,11,Coyhaique,111,Lago Verde,11102,494,358,852


In [3]:
# Importa datos de comunas corregidas por establecimiento de salud

df_establecimiento = pd.read_excel('data/establecimientos_salud_dict.xlsx') #Archivo ubicado dentro de la carpeta data


df_establecimiento.head()

,establecimiento_salud,comuna_seremi_xi,provincia_seremi_xi
0,Actividades gestionadas por la Dirección del S...,Coyhaique,Coyhaique
1,Centro Clínico Militar Coyhaique,Coyhaique,Coyhaique
2,Centro Comunitario de Salud Familiar Villa Ma...,Aysen,Aysen
3,Centro Comunitario de Salud Familiar Alejandro...,Coyhaique,Coyhaique
4,Centro Comunitario de Salud Familiar Puerto Ch...,Aysen,Aysen


In [4]:
# Importa datos desde la base de datos recivida via FTP
## PRIMERO: Guardar archivo dentro de la carpeta data
## SEGUNDO: No cambiar nombre ni formato del archivo recibido
## TERCERO: RECORDAR CAMBIAR FECHA EN EL ARCHIVO A IMPORTAR A CONTINUACION, i.e: 20210829 para este domingo 29 de Agosto 2021

df =  pd.read_csv('data/20210907_Aisen_notificaciones_0600.csv', 
                   sep='~', # CSV con separador ''~''
                   low_memory=False
                  )

In [5]:
df_trazabilidad =  pd.read_csv('data/20210907_Región de Aysén del General Carlos Ibáñez del Campo_Seguimiento.csv', 
                   sep=';', # CSV con separador ''~''
                   low_memory=False
                  )

##  limpieza y ordenamiento 

In [ ]:
# Une datos con diccionario comunal

dff = df.merge(df_establecimiento, how='left')

In [ ]:
# Integra datos poblacionales

df_merged = dff.merge(df_poblacion.drop(columns= ['Unnamed: 0','orden','nombre_region', 'codigo_region', 'nombre_provincia','codigo_provincia']), left_on='comuna_seremi_xi', right_on='nombre_comuna', how='left' )

In [ ]:
# Filtra datos obtenidos solo para pacientes con residencia en la Región de Aysén

data_aysen = df_merged[df_merged['region_residencia'] == 'Región de Aysén del General Carlos Ibáñez del Campo']

data_aysen.head(1)

In [ ]:
data_aysen_filtered = data_aysen[(data_aysen['estado_caso']!='No validada')&(
                          data_aysen['vigente_no_eliminado']==True)&(
                          data_aysen['seremi']=='SEREMI De Aisén del General Carlos Ibañez del Campo')
                       ].rename(columns={'edad_hombres':'pop_hombres',
                                         'edad_mujeres':'pop_mujeres',
                                         'total':'pop_total'})

data_aysen_filtered.sample(1)

In [ ]:
# Transforma Fechas a formato DateTime 

data_aysen_filtered['fecha_notificacion'] = pd.to_datetime(data_aysen_filtered['fecha_notificacion'], errors = 'coerce') 

data_aysen_filtered['fecha_toma_muestra_1'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_1'], errors = 'coerce')

data_aysen_filtered['fecha_toma_muestra_2'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_2'], errors = 'coerce')

data_aysen_filtered['fecha_toma_muestra_3'] = pd.to_datetime(data_aysen_filtered['fecha_toma_muestra_3'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_1'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_1'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_2'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_2'], errors = 'coerce')

data_aysen_filtered['fecha_resultado_pcr_3'] = pd.to_datetime(data_aysen_filtered['fecha_resultado_pcr_3'], errors = 'coerce')

In [ ]:
# Muestra variables presente en DB en formato DF

s=pd.DataFrame(data_aysen_filtered.columns)

s[s[0]=='tipo_seguimiento']


In [ ]:
# Selecciona variables de importancia

data_aysen_clean =  data_aysen_filtered[[
                                'numero_folio',
                                'comuna_residencia',
                                'fecha_notificacion',
                                'semana_epidemiologica',
                                'etapa_clinica',
                                'tipo_caso_busqueda',
                                'establecimiento_salud',
                                'resultado_pcr_1',
                                'resultado_pcr_2',
                                'resultado_pcr_3',
                                'fecha_toma_muestra_1',
                                'fecha_toma_muestra_2',
                                'fecha_toma_muestra_3',
                                'fecha_resultado_pcr_1',
                                'fecha_resultado_pcr_2',
                                'fecha_resultado_pcr_3',
                                'estado_paciente',
                                'lugar_reposo',
                                'estado_caso',
                                'presentacion_clinica',
                                'contacto_caso_sospechoso',
                                'motivo_examen',
                                'hospitalizacion',
                                'trabajador_salud',
                                'caso_cerrado',
                                'fecha_primeros_sintomas',
                                'fecha_diagnostico',
                                'comuna_seremi_xi',
                                'provincia_seremi_xi',
                                'sexo',
                                'edad',
                                'pop_hombres',
                                'pop_mujeres',
                                'pop_total'
                               ]]

In [ ]:
# Agrega 53 semanas a las semanas de 2021 y crea semana_epidemiologica_21

data_aysen_clean1 = data_aysen_clean[data_aysen_clean['fecha_notificacion']<=pd.to_datetime('2021-01-02')]
data_aysen_clean2 = data_aysen_clean[data_aysen_clean['fecha_notificacion']>=pd.to_datetime('2021-01-03')]


data_aysen_clean1['semana_epidemiologica_21'] = data_aysen_clean1['semana_epidemiologica']
data_aysen_clean2['semana_epidemiologica_21'] = data_aysen_clean2['semana_epidemiologica']+53

data_aysen_clean_21 = data_aysen_clean1.append(data_aysen_clean2, ignore_index=True)


In [ ]:
data_aysen_clean.sample(10)

In [ ]:
df_seguimiento = pd.read_csv('data/df_seguimiento_full.csv',
                            sep=';',
                             low_memory=False
                            )

In [ ]:
df_seguimiento

In [ ]:
# Divide strings utilizando regex para extraer nfolio de casos, 
# Separa nfolio caso indice de nfolio contacto estrecho y guarda caso índice
# Y extrae datos de PNTM

df_seguimiento['n_folio'].str.extract(r'([0-9]+)|SC[0-9]+-([0-9]+)|PNTM-([0-9]+)', flags=0,expand=True).rename(columns={0:'nfolio_caso',
                                                                                                          1:'nfolio_caso_indice',
                                                                                                          2:'nfolio_pntm'
                                                                                                        })

In [ ]:
# Separa nfolio caso índice de nfolio contacto estrecho y guarda contacto estrecho

df_seguimiento['n_folio'].str.extract(r'SC([0-9]+)-', 
                                      flags=0,
                                      expand=True).rename(
                                      columns={0:'nfolio_contacto_estrecho'})

In [ ]:
# Realiza merge con DF original utilizando index de importación
df_seguimiento_merged = df_seguimiento.merge(df_seguimiento['n_folio'].str.extract(
                                                 r'([0-9]+)|SC[0-9]+-([0-9]+)|PNTM-([0-9]+)', 
                                                 flags=0,
                                                 expand=True).rename(
                                                 columns={0:'nfolio_caso',
                                                          1:'nfolio_caso_indice',
                                                          2:'nfolio_pntm'}), 
                                             how='left',
                                             left_index= True,
                                             right_index=True
                                      ).merge(df_seguimiento['n_folio'].str.extract(
                                                                 r'SC([0-9]+)-', 
                                                                 flags=0,
                                                                 expand=True).rename(
                                                                 columns={0:'nfolio_contacto_estrecho'}), 
                                                                 how='left',
                                                                 left_index= True,
                                                                 right_index=True
                                      )

df_seguimiento_merged.sample(3)

In [ ]:
df_seguimiento_merged.head()

In [ ]:
# Revisa folio en DB de notificaciones

data_aysen_clean[data_aysen_clean['numero_folio']==14832232]

In [ ]:
# Todos los datos poseen un folio 

df_seguimiento_merged[df_seguimiento_merged['n_folio'].isnull()]

In [ ]:
df_seguimiento_merged.columns

In [ ]:
pd.DataFrame(df_seguimiento_merged.columns).rename(columns={0:'variables_seguimiento'}).transpose()

In [ ]:
df_seguimiento_merged['folio_se_convierte_en_caso']

In [ ]:
# Revisa existencia de una variable dentro del DF y sus respectivos valores

df_seguimiento_merged['folio_se_convierte_en_caso'].value_counts()


# Variables de interes


|Variable|Tipo|Observación|
|---|---|---|
|`n_folio`|Text|folio generado por Número y Letras|
|`nfolio_caso`|Numeric|folio de caso registrado|
|`nfolio_caso_indice`|Numeric|folio de caso indice registrado|
|`nfolio_pntm`|Numeric|folio plataforma nacional de toma de muestra|
|`nfolio_contacto_estrecho`|Numeric|folio de contacto estrecho|
|`tipo_seguimiento`|Text|contacto, caso, no notificado indice|
|`dia_contacto`|DateTime|Fecha de contacto |
|`fecha_notificacion`|DateTime|Fecha de notificación|
|`nombre_institucion_indice`|Text|Nombre institución desde donde se deriva el caso|
|`region_institucion_indice`|Numeric| Región d ela cual se deriva el caso|
|`nombre_region_institucion_indice`|Text|Nombre de región de la institución índice|
|`nombre_institucion_seguimiento`|Text|Nombre de la institución encargada de realizar seguimiento|
|`region`|Text|Región en la cual se realiza el seguimiento|
|`id_region`|Numeric|Número de la región de seguimiento|
|`fecha_cumple_requisitos`|DateTime|Fecha en la cual cumple requisitos para seguimiento|
|`TTA`|Text|APS o SEREMI|
|`cumple_aislamiento_cuarentena`|Boolean|Sin datos, t, f|
|`folio_se_convierte_en_caso`|Text|Número de folio o Sin Datos|
|`visible_no_eliminado`|Boolean|Sin datos, t, f|
|`suspendido`|Text|egresado|
|`sintomas`|Text|Sintomatología, Sin datos o Ninguno|
|`observacion_seguimiento_contactos`|Text| Escrito por el trazador|
|`tiene_resultado_covid`|Text|Positivo, Negativo, No concluyente, Sin Datos|
|`derivacion_a_su`|Boolean| t f Sin datos|
|`derivacion_toma_muestra`|Boolean|t f Sin datos|
|`fecha_derivacion_toma_muestra`|Text|Fecha Sin datos|
|`cont_fecha_seguimiento`|Text|Texto o Sin Datos|
|`cont_tipo_contacto`|Text|contacto_familiar, contacto_social, contacto_institucional, contacto_laboral|
|||contacto_pasajero, contacto_sala_espera, contacto_personal_salud,Datos|
|`cont_parentesco`|Text|Hijo/a, Madre/Padre, Otro familiar, Pareja, Hermano/a, Otra relación/No familiar, Sin Datos|
|`cont_otra_relacion`|Text|Sin Datos, otro, hijo de cuidadora, otro adulto mayor en cuidado, cuñado. sobrino de cuidadora, AMIGO,  Amigo|
||| pareja del tío de su madre,cuidadora, PAREJA PADRE, otro adulto en cuidados, Pareja de mamá, Hijastro|
|`cont_tipo_transporte`|Text|Aéreo, Sin Datos, Terrestre|
|`cont_nombre_vuelo`|Text|Sin Datos, JETSMART JA245 CCP - BALMACEDA, SKY 433, COPA CM111, 
|||SKY 433 STGO- COYHAIQUE 21-08-2021 / 11 AM, SK, DL147 ATLANTA - STGO, JETSMART CCP-BALMACEDA, LA 272 |
|||JETSMART JA245, VUELO CM111, LA272, JETSMART JA 245 CCP - BALMACEDA|                 
|`cont_fecha_llegada`|Text|Sin Datos, 2021-07-21, 2021-08-21, 2021-07-01, 2021-07-03, 2021-07-09, 2021-07-02|
|`detalle_pasajero`|Text|Observaciones adicionales de viaje|
|`cont_inicio_cuarentena`|DateTime|Fecha de inicio de cuarentena|
|`cont_fin_cuarentena`|DateTime|Fecha y Sin Datos|
|`cont_requiere_licencia_medica`|Boolean|Sin Datos, NO, SI, Seleccione|
|`cont_se_emite_licencia`|Boolean|Sin Datos, t, f|
|`cont_fecha_licencia`|DateTime|Fecha de licencia|
|`requiere_canasta_alimentos`|Boolean|Sin Datos, t, f|
|`cantidad_personas_vivienda`|Numeric|Integrantes familiares de 0 a 6|
|`cont_estado_seguimiento`|Text|Aún no contactado, En proceso de seguimiento, Se transforma en caso, Contacto sintomático, No localizado|
|`fecha_creacion`|DateTime|Fecha de creación|
|`tipo_del_contacto`|Text|Sin Datos, Llamada telefónica, Visita domiciliaria|
|`contacto_localizado`|Boolean|Sin Datos, t, f|
|`empresa`|Text|Información de Empresa de trabajo|
|`institucion`|Text|Información de la institución|

## Limpieza y ordenamiento de datos



Segun variables seleccionadas como de importancia de análisis y obsrvaciones a filtrar

In [ ]:
# Transforma fecha_cumple_requisitos a formato DateTime64 

df_seguimiento_merged['fecha_cumple_requisitos'] = pd.to_datetime(df_seguimiento_merged['fecha_cumple_requisitos'], errors='coerce')
df_seguimiento_merged['fecha_notificacion'] = pd.to_datetime(df_seguimiento_merged['fecha_notificacion'], errors='coerce')
df_seguimiento_merged['fecha_creacion'] = pd.to_datetime(df_seguimiento_merged['fecha_creacion'], errors='coerce')

#Crea diferencia de fecha
df_seguimiento_merged['dif_fecha'] = df_seguimiento_merged['fecha_cumple_requisitos'] - df_seguimiento_merged['fecha_notificacion']

In [ ]:
# Filtra los timedelta mayores a 14 dias

seg = df_seguimiento_merged[df_seguimiento_merged['dif_fecha']<pd.to_timedelta('14 days')]

In [ ]:
#Filtra visible_no_eliminado y bota duplicados en n_folio

aux = seg[seg['visible_no_eliminado']=='t'].drop_duplicates('n_folio')

In [ ]:
# Filtra tipo_seguimiento para los casos 

aux_caso = aux[aux['tipo_seguimiento']=='caso']

In [ ]:
aux_contacto = aux[aux['tipo_seguimiento']=='contacto']
#Crea columna dif_fecha_contacto
aux_contacto['dif_fecha_contacto'] = aux_contacto['fecha_creacion']-aux_contacto['fecha_notificacion']

aux_contacto = aux_contacto[aux_contacto['dif_fecha_contacto']>=pd.to_timedelta(-14)]

In [ ]:
aux_contacto_nuevos = aux[aux['tipo_seguimiento']=='contacto']
aux_contacto_nuevos = aux_contacto_nuevos[(aux_contacto_nuevos['suspendido']=='activo')|(aux_contacto_nuevos['suspendido']=='egresado')]


In [ ]:
"""aux_contacto_viejos = aux_contacto[(aux_contacto['fecha_creacion']-pd.to_timedelta('7 days'))]

aux_contacto_viejos= aux_contacto_viejos[(aux_contacto_viejos['suspendido']=='activo')|(aux_contacto_viejos['suspendido']=='egresado')]"""


In [ ]:
#aux_contacto[
(aux_contacto['fecha_creacion']-pd.to_timedelta('7 days'))
#]

In [ ]:
aux_contacto['fecha_creacion']-pd.to_timedelta(-7)

In [ ]:
aux_contacto['fecha_creacion']

In [ ]:
df_seguimiento_merged['visible_no_eliminado']

In [ ]:
data_aysen_clean

In [ ]:
df_seguimiento_filtered = df_seguimiento_merged

In [ ]:
# Revisa tipo de datos
df_seguimiento_filtered['nfolio_caso'].dtype

In [ ]:
# Revisa tipo de datos para merging

data_aysen_clean['numero_folio'].dtype

In [ ]:
# Transforma dtype de ambas columnas a float64 para permitir merging 

df_seguimiento_filtered['nfolio_caso']= df_seguimiento_filtered['nfolio_caso'].astype('float64', errors='raise')


data_aysen_clean['numero_folio']= data_aysen_clean['numero_folio'].astype('float64', errors='raise')

In [ ]:
# Combina ambas columnas para nfolio_caso y nfolio_contacto_estrecho y 
# agrega NA en donde corresponda a otros casos como PNTM o nfolio_caso_indice

df_seguimiento_filtered['nfolio_merge'] = df_seguimiento_filtered['nfolio_caso'].combine_first(df_seguimiento_filtered['nfolio_contacto_estrecho']).astype('float64', errors='raise')



In [ ]:
# Realiza merge con base de datos de notificaciones agregando los valores de casos confirmados y probables


df_seguimiento_filtered= df_seguimiento_filtered.merge(data_aysen_clean[[
                                            'numero_folio',
                                            'etapa_clinica',
                                            'tipo_caso_busqueda',
                                            'establecimiento_salud',
                                            'resultado_pcr_1',
                                            'resultado_pcr_2',
                                            'resultado_pcr_3',
                                            'fecha_resultado_pcr_1',
                                            'fecha_resultado_pcr_2',
                                            'fecha_resultado_pcr_3',
                                            'comuna_seremi_xi',
                                            'provincia_seremi_xi',
                                            'edad',
                                            'sexo',
                                            'pop_hombres',
                                            'pop_mujeres',
                                            'pop_total'
                                        ]], left_on='nfolio_merge', right_on='numero_folio' , how='left')

In [ ]:
# Selecciona variables de importancia

df_seguimiento_clean = df_seguimiento_filtered[[
    'n_folio',
    'nfolio_caso',
    'nfolio_caso_indice',
    'nfolio_pntm',
    'nfolio_contacto_estrecho',
    'tipo_seguimiento',
    'dia_contacto',
    'fecha_notificacion',
    'nombre_institucion_indice',
    'region_institucion_indice',
    'nombre_region_institucion_indice',
    'region',
    'id_region',
    'fecha_cumple_requisitos',
    'TTA',
    'cumple_aislamiento_cuarentena',
    'folio_se_convierte_en_caso',
    'visible_no_eliminado',
    'suspendido',
    'sintomas',
    'observacion_seguimiento_contactos',
    'tiene_resultado_covid',
    'derivacion_a_su',
    'derivacion_toma_muestra',
    'fecha_derivacion_toma_muestra',
    'cont_fecha_seguimiento',
    'cont_tipo_contacto',
    'cont_parentesco',
    'cont_otra_relacion',
    'cont_tipo_transporte',
    'cont_nombre_vuelo',
    'cont_fecha_llegada',
    'detalle_pasajero',
    'cont_inicio_cuarentena',
    'cont_fin_cuarentena',
    'cont_requiere_licencia_medica',
    'cont_se_emite_licencia',
    'cont_fecha_licencia',
    'requiere_canasta_alimentos',
    'cantidad_personas_vivienda',
    'cont_estado_seguimiento',
    'fecha_creacion',
    'tipo_del_contacto',
    'contacto_localizado',
    'empresa',
    'institucion',
    'etapa_clinica',
    'tipo_caso_busqueda',
    'establecimiento_salud',
    'resultado_pcr_1',
    'resultado_pcr_2',
    'resultado_pcr_3',
    'fecha_resultado_pcr_1',
    'fecha_resultado_pcr_2',
    'fecha_resultado_pcr_3',
    'comuna_seremi_xi',
    'provincia_seremi_xi',
    'edad',
    'sexo',
    'pop_hombres',
    'pop_mujeres',
    'pop_total'
]]

In [ ]:
df_seguimiento_clean

In [ ]:
df_seguimiento_clean['nfolio_caso_indice'].value_counts()

In [ ]:
#Se consideran solo los visible no eliniminados y se botan duplicados

df_seguimiento_clean= df_seguimiento_clean[df_seguimiento_clean['visible_no_eliminado']=='t'].drop_duplicates()


In [ ]:
df_seguimiento_clean['nfolio_caso_indice'].value_counts()

# Indicador 5: Oportunidad en la investigación epidemiológica de casos

## Objetivo: Fortalecer la investigación epidemiológica de los casos

### Meta: >90%

**Periodicidad:** Semanal

**Escala Territorial:** Comunal, Regional, Nacional

**Fuente de Información:** Base EPIVIGILIA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento

### Cálculo del indicador a nivel comunal

$\frac{No.\:casos\:nuevos\:registrados\:investigados\:dentro\:de\:48\:hrs\:por\:comuna\:1er\:establecimiento,\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:comuna\:1er\:establecimiento\:semana\:x}x100$

### Cálculo del indicador a nivel regional

$\frac{No.\:casos\:nuevos\:registrados\:investigados\:dentro\:de\:48\:hrs\:por\:región\:1er\:establecimiento,\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:región\:1er\:establecimiento\:semana\:x}x100$

**Observación**

- No se incluyen los casos nuevos no notificados
- Se considera un caso nuevo registrado a aquel caso notificado que cumple los requisitos para estar en el modulo de seguimiento
- La fecha para determinar el *inicio de seguimiento* corresponde a la **fecha de resultado (caso confirmado)** o **fecha de notificación (caso probable)**. Para un caso confirmado, si el resultado del test es anterior a la fecha de notificación, se debe iniciar seguimiento a partir de la fecha en que el registro está disponible en la bandeja de seguimiento.
- Se cumple el indicador si, dentro de las 48 hrs. a partir de la fecha de inicio de seguimiento se tiene al menos una actividad de seguimiento exitosa (se logra ubicar al caso).
- El primer establecimiento corresponde al establecimiento dado por API FONASA. Si el registro no hace match con API FONASA, se considera el establecimiento de notificación.
- Se consideran sólo registros de casos que estén activos o egresados en la bandeja de seguimiento.

In [ ]:
df_seguimiento_clean['fecha_creacion'] #casos nuevos registrados

In [ ]:

#Se consideran sólo registros de casos que estén activos o egresados en la bandeja de seguimiento.

df_seguimiento_ind5= df_seguimiento_clean[(df_seguimiento_clean['suspendido']=='activo')|(df_seguimiento_clean['suspendido']=='egresado')]

df_seguimiento_ind5.sample(2)

In [ ]:
df_seguimiento_ind5[['suspendido', 'etapa_clinica', 'visible_no_eliminado','resultado_pcr_1','resultado_pcr_2','resultado_pcr_3']]

In [ ]:
# Aún existen casos con 'etapa_clinica' DESCARTADA y SOSPECHA 

df_seguimiento_ind5.etapa_clinica.value_counts()

In [ ]:

df_seguimiento_ind5[df_seguimiento_ind5['etapa_clinica']=='CONFIRMADA'][
    ['etapa_clinica',
     'tipo_seguimiento',
     'fecha_cumple_requisitos', 
     'fecha_notificacion',
     'fecha_resultado_pcr_1',
     'fecha_resultado_pcr_2',
     'fecha_resultado_pcr_3']
]

In [ ]:
# Elimina etapa_clinica DESCARTADA y SOSPECHA
df_seguimiento_ind5 = df_seguimiento_ind5[(df_seguimiento_ind5['etapa_clinica']!='DESCARTADA')|(df_seguimiento_ind5['etapa_clinica']!='SOSPECHA')]

In [ ]:
# Toma en cuenta solo el primer contacto con el contacto o caso

df_seguimiento_ind5= df_seguimiento_ind5.sort_values(by=['n_folio', 'dia_contacto']).groupby('n_folio').head(1)

In [ ]:
#Crea diferencia de fecha
df_seguimiento_ind5['dif_fecha'] = df_seguimiento_ind5['fecha_cumple_requisitos'] - df_seguimiento_ind5['fecha_notificacion']

In [ ]:
# Crea columna considerando si diferencial de fecha cumple requisitos y fecha de notificacion
# es menor a un timedelta de d 2 días

df_seguimiento_ind5['ind5'] = df_seguimiento_ind5['dif_fecha']<= pd.to_timedelta('2 days')

In [ ]:
df_seguimiento_ind5[['fecha_creacion',
                       'fecha_notificacion', 
                       'fecha_cumple_requisitos', #Se considera un caso nuevo registrado a aquel caso notificado que cumple los requisitos para estar en el modulo de seguimiento
                       'fecha_derivacion_toma_muestra',
                       'cont_fecha_seguimiento',
                       'cont_inicio_cuarentena',
                       'cont_fin_cuarentena',
                       'cont_fecha_licencia',
                       'dif_fecha'
                      ]].sample(4)

In [ ]:
df_seguimiento_ind5.groupby(['fecha_cumple_requisitos', 'ind5']).count()[['n_folio']].reset_index()

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_seguimiento_ind5.groupby(['fecha_cumple_requisitos', 'ind5']).count()[['n_folio']].reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color= 'ind5',
        color_discrete_map={
            True: '#750D86',
            False: '#EB663B',#rgb(222,203,228)#C075A6
            
          },
        labels={'ind5':'Cumple'}, 
       ).update_layout(title= 'Ind:5 Oportunidad en la investigación epidemiológica de casos',
                    yaxis_title='No. casos nuevos investigados en 48 hrs',
                    xaxis_title='Fecha',
                    template='simple_white',    ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0.1, 
                                      opacity=0.8
                                     )                     
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_seguimiento_ind5.groupby(['fecha_cumple_requisitos', 'ind5']).count()[['n_folio']].reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby(['semana_epidemiologica_21','ind5']).mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        color= 'ind5',
        color_discrete_map={
            True: '#750D86',
            False: '#EB663B',#rgb(222,203,228)#C075A6
            
          },
        labels={'ind5':'Cumple'}, 
       ).update_layout(title= 'Ind:5 Oportunidad en la investigación epidemiológica de casos',
                    yaxis_title='No. casos nuevos investigados en 48 hrs',
                    xaxis_title='Semana epidemiológica',
                    template='simple_white',    ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0.1, 
                                      opacity=0.8
                                     )                     
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
df_ind5= df_seguimiento_ind5.groupby(['fecha_cumple_requisitos', 'ind5']).count()[['n_folio']].reset_index()

In [ ]:
df_ind5_perc = (df_ind5[df_ind5['ind5']==True].set_index('fecha_cumple_requisitos')[['n_folio']]/df_ind5.groupby('fecha_cumple_requisitos').sum('n_folio')[['n_folio']])*100

In [ ]:
px.scatter(df_ind5_perc.reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color='n_folio',
        color_continuous_scale=px.colors.sequential.Viridis_r,
        labels={'n_folio':'Oportunidad (%)'},
        ).update_layout(title= 'Ind:5 Oportunidad en la investigación epidemiológica de casos',
                       yaxis_title='No. casos nuevos investigados en 48 hrs (%)',
                       xaxis_title='Fecha',
                       template='simple_white',
                 #      xaxis=dict(tickmode = 'array',
                  #                tickvals = list(range(12,88,3)),
                   #               ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                    #             ),
                 #   ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                  #             ).add_hline(y=10, 
                   #                        line_width=3, 
                    #                       line_dash='longdash', 
                     #                      line_color='olive',
                      #                     annotation_text='Meta nacional (10%)',
                       #                    annotation_position='bottom right'
                        #         ).add_hline(y=mean_bac_ind, 
                         #                    line_width=3, 
                          #                   line_dash='longdash', 
                           #                  line_color='green',
                            #                 annotation_text= 'Promedio BAC ('+str(mean_bac_ind)+'%)',
                             #                annotation_position='bottom right'
                              #       ).add_vrect(x0='51', x1='50', 
                               #            annotation_text='2020', 
                                #           annotation_position='top right',
                                 #          annotation=dict(font_size=18, 
                                  #                         font_family="'Open Sans', verdana, arial, sans-serif"),
                                   #                        opacity=0, 
                                    #                       line_width=0
                                     #     ).add_vrect(x0='84', x1='86', 
                                      #     annotation_text='2021', 
                                       #    annotation_position='top right',
                                        #   annotation=dict(font_size=18, 
                                         #                  font_family="'Open Sans', verdana, arial, sans-serif"),
                                          #                 opacity=0, 
                                           #                line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

In [ ]:
df_ind5_perc.reset_index().head()

In [ ]:
data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']]

In [ ]:
df_ind5_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean()

In [ ]:
mean_ind5 = df_ind5_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean()['n_folio'].mean()

In [ ]:
px.scatter(df_ind5_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        #trendline='rolling',
        #trendline_options=dict(window=5),
        color='n_folio',
        color_continuous_scale=px.colors.sequential.Viridis_r,
        labels={'n_folio':'Oportunidad (%)'},
        ).update_layout(title= 'Ind5: Oportunidad en la investigación epidemiológica de casos',
                       yaxis_title='No. casos nuevos investigados en 48 hrs (%)',
                       xaxis_title='Semana epidemiológica',
                       yaxis_range= [50,105],
                       template='simple_white',
                       xaxis=dict(tickmode = 'array',
                                  tickvals = list(range(12,88,3)),
                                  ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                                 ),
                    ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                              ).add_hline(y=90,
                                            line_width=3, 
                                            line_dash='longdash', 
                                            line_color='#222A2A',
                                            annotation_text='Meta (90%)',
                                            annotation_position='bottom right'
                                # ).add_hline(y=mean_ind5, 
                                 #            line_width=3, 
                                  #           line_dash='longdash', 
                                   #          line_color='#0D2A63',
                                    #         annotation_text= 'Promedio ('+str(round(mean_ind5, 2))+'%)',
                                     #        annotation_position='bottom right'
                                     ).add_vrect(x0='51', x1='53', 
                                           annotation_text='2020', 
                                           annotation_position='inside right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                          ).add_vrect(x0='84', x1='88', 
                                           annotation_text='2021', 
                                           annotation_position='inside right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

# Indicador 6: Capacidad de investigación de contactos

## Objetivos: Fortalecer la investigación epidemiológica de los contactos estrechos

### Meta: >80%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento

**Cálculo del indicador a nivel comunal**

$\frac{No.\:casos\:nuevos\:con\:contactos\:estrechos\:por\:comuna\:1er\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:por\:comuna\:1er\:establecimiento\:semana\:x}$

**Cálculo del indicador a nivel regional**

$\frac{No.\:casos\:nuevos\:con\:contactos\:estrechos\:por\:región\:1er\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:por\:región\:1er\:establecimiento\:semana\:x}$

**Observaciones**

- No se incluyen los casos nuevos no notificados
- Se considera un caso nuevo registrado a aquel caso nuevo notificado que cumple los requisitos para estar en el módulo de seguimiento
- Los casos nuevos con contactos estrechos son considerados de forma dicotómica (sí o no)
- El primer establecimiento corresponde al establecimiento de notificación. Se seguirá considerando el primer establecimiento para este indicador, aún si se derivó previo a la investigación epidemiológica.
- Se consideran sólo registros activos o egresados en la bandeja de seguimiento

In [ ]:
df_seguimiento_ind6 = df_seguimiento_clean

In [ ]:
df_seguimiento_ind6['is_contact']= df_seguimiento_ind6['nfolio_contacto_estrecho'].notna()

In [ ]:
# Crea una lista con casos indice

casos_indice = df_seguimiento_ind6[df_seguimiento_ind6['nfolio_caso_indice'].notna()]['nfolio_caso_indice'].drop_duplicates().astype('float64').to_list()



In [ ]:
# Revisa si un caso es caso indice

df_seguimiento_ind6['is_index'] = df_seguimiento_ind6['nfolio_caso'].isin(casos_indice)

In [ ]:
# Crea columna para mostrando si es o caso indice o contacto estrecho

df_seguimiento_ind6['is_index_or_contact'] = df_seguimiento_ind6['is_index'] | df_seguimiento_ind6['is_contact']

In [ ]:
df_seguimiento_ind6['is_index_or_contact'].value_counts()

In [ ]:
df_seguimiento_ind6_clean = df_seguimiento_ind6[(df_seguimiento_ind6['suspendido']=='activo')|(df_seguimiento_ind6['suspendido']=='egresado')]

In [ ]:
# Elimina etapa_clinica DESCARTADA y SOSPECHA
df_seguimiento_ind6_clean = df_seguimiento_ind6_clean[(df_seguimiento_ind6_clean['etapa_clinica']!='DESCARTADA')|(df_seguimiento_ind6_clean['etapa_clinica']!='SOSPECHA')]

In [ ]:
df_seguimiento_ind6_clean['nfolio_caso_indice'].value_counts()

In [ ]:
df_seguimiento_ind6_clean.head(1)

In [ ]:
# Toma en cuenta solo el primer contacto con el contacto o caso

df_seguimiento_ind6_clean= df_seguimiento_ind6_clean.sort_values(by=['n_folio', 'dia_contacto']).groupby('n_folio').head(1)

In [ ]:
df_seguimiento_ind6_clean.groupby(['fecha_cumple_requisitos', 'is_index_or_contact']).count()[['n_folio']].reset_index()

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_seguimiento_ind6_clean.groupby(['fecha_cumple_requisitos', 'is_index_or_contact']).count()[['n_folio']].reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color= 'is_index_or_contact',
        color_discrete_map={
            True: '#750D86',
            False: '#EECA3B',#rgb(222,203,228)#C075A6
          },
        labels={'is_index_or_contact':'Con contactos'}, 
       ).update_layout(title= 'Ind 6: Capacidad de investigación de contactos estrechos de la Región de Aysén',
                    yaxis_title='No. casos nuevos con contactos estrechos',
                    xaxis_title='Fecha',
                    template='simple_white',
                      
                      ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0, 
                                      opacity=1
                                     )                
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_seguimiento_ind6_clean.groupby(['fecha_cumple_requisitos', 'is_index_or_contact']).count()[['n_folio']].reset_index().merge(
    data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby(['semana_epidemiologica_21','is_index_or_contact']).mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        color= 'is_index_or_contact',
        color_discrete_map={
            True: '#750D86',
            False: '#EECA3B',#rgb(222,203,228)#C075A6
          },
        labels={'is_index_or_contact':'Con contactos'}, 
       ).update_layout(title= 'Ind 6: Capacidad de investigación de contactos estrechos de la Región de Aysén',
                    yaxis_title='No. casos nuevos con contactos estrechos',
                    xaxis_title='Semana epidemiológica',
                    template='simple_white',    ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0.1, 
                                      opacity=0.8
                                     )                     
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
df_ind6= df_seguimiento_ind6_clean.groupby(['fecha_cumple_requisitos', 'is_index_or_contact']).count()[['n_folio']].reset_index()

In [ ]:
df_ind6_perc = (df_ind6[df_ind6['is_index_or_contact']==True].set_index('fecha_cumple_requisitos')[['n_folio']]/df_ind6.groupby('fecha_cumple_requisitos').sum('n_folio')[['n_folio']])*100

In [ ]:
px.scatter(df_ind6_perc.reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color='n_folio',
        color_continuous_scale=px.colors.sequential.Magma_r,
        labels={'n_folio':'Con contactos (%)'},
        ).update_layout(title= 'Ind 6: Capacidad de investigación de contactos estrechos de la Región de Aysén',
                       yaxis_title='No. casos nuevos con contactos estrechos (%)',
                       xaxis_title='Fecha',
                       template='simple_white',
                 #      xaxis=dict(tickmode = 'array',
                  #                tickvals = list(range(12,88,3)),
                   #               ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                    #             ),
                 #   ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                  #             ).add_hline(y=10, 
                   #                        line_width=3, 
                    #                       line_dash='longdash', 
                     #                      line_color='olive',
                      #                     annotation_text='Meta nacional (10%)',
                       #                    annotation_position='bottom right'
                        #         ).add_hline(y=mean_bac_ind, 
                         #                    line_width=3, 
                          #                   line_dash='longdash', 
                           #                  line_color='green',
                            #                 annotation_text= 'Promedio BAC ('+str(mean_bac_ind)+'%)',
                             #                annotation_position='bottom right'
                              #       ).add_vrect(x0='51', x1='50', 
                               #            annotation_text='2020', 
                                #           annotation_position='top right',
                                 #          annotation=dict(font_size=18, 
                                  #                         font_family="'Open Sans', verdana, arial, sans-serif"),
                                   #                        opacity=0, 
                                    #                       line_width=0
                                     #     ).add_vrect(x0='84', x1='86', 
                                      #     annotation_text='2021', 
                                       #    annotation_position='top right',
                                        #   annotation=dict(font_size=18, 
                                         #                  font_family="'Open Sans', verdana, arial, sans-serif"),
                                          #                 opacity=0, 
                                           #                line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

In [ ]:
mean_ind6 = df_ind6_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean()['n_folio'].mean()

In [ ]:
px.scatter(df_ind6_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        #trendline='rolling',
        #trendline_options=dict(window=5),
        color='n_folio',
        color_continuous_scale=px.colors.sequential.Magma_r,
        labels={'n_folio':'Con contactos (%)'},
        ).update_layout(title= 'Ind 6: Capacidad de investigación de contactos estrechos de la Región de Aysén',
                       yaxis_title='No. casos nuevos con contactos estrechos (%)',
                       xaxis_title='Semana epidemiológica',
                       yaxis_range= [0,100],
                       template='simple_white',
                       xaxis=dict(tickmode = 'array',
                                  tickvals = list(range(12,88,3)),
                                  ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                                 ),
                    ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                              ).add_hline(y=80,
                                            line_width=3, 
                                            line_dash='longdash', 
                                            line_color='#222A2A',
                                            annotation_text='Meta (80%)',
                                            annotation_position='bottom right'
                                 ).add_hline(y=mean_ind6, 
                                             line_width=3, 
                                             line_dash='longdash', 
                                             line_color='#750D86',
                                             annotation_text= 'Promedio ('+str(round(mean_ind6, 2))+'%)',
                                             annotation_position='bottom right'
                                     ).add_vrect(x0='51', x1='53', 
                                           annotation_text='2020', 
                                           annotation_position='top right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                          ).add_vrect(x0='84', x1='88', 
                                           annotation_text='2021', 
                                           annotation_position='top right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

# Indicador 7: Capacidad de investigación de contactos en 48h


## Objetivo: Evaluar la capacidad de seguimiento de contactos

### Meta: >80%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento.

**Cálculo del indicador a nivel comunal**

$\frac{No.\:contactos\:identificados\:investigados\:dentro\:de\:48hrs\:por\:comuna\:1er\:establecimiento\:semana\:x}{No.\:contactos\:identificados\:por\:comuna\:1er\:establecimiento\:semana\:x}x100$

**Cálculo del indicador a nivel regional**

$\frac{No.\:contactos\:identificados\:investigados\:dentro\:de\:48hrs\:por\:región\:1er\:establecimiento\:semana\:x}{No.\:contactos\:identificados\:por\:región\:1er\:establecimiento\:semana\:x}x100$

**Observaciones**

- No se incluyen los contactos nuevos no notificados
- Se considera como registro de contacto aquel que cumple las condiciones para estar en la bandeja de seguimiento
- Se consideran sólo registros de casos activos o egresados en la bandeja de seguimiento
- El primer establecimiento corresponde al establecimiento dado por API FONASA. Si el registro no hace match con API FONASA, se considera el establecimiento para este indicador, aún si se derivó previo a la investigación epidemiológica

In [ ]:
df_seguimiento_ind7 = df_seguimiento_clean

In [ ]:
df_seguimiento_ind7= df_seguimiento_ind7[df_seguimiento_ind7['nfolio_contacto_estrecho'].notna()]#[['fecha_cumple_requisitos','dia_contacto']]

In [ ]:
# Filtra contactos que hayan sido localizados

df_seguimiento_ind7 = df_seguimiento_ind7[df_seguimiento_ind7['contacto_localizado']=='t']

In [ ]:
# Se consideran sólo registros de casos activos o egresados
df_seguimiento_ind7_clean = df_seguimiento_ind7[(df_seguimiento_ind7['suspendido']=='activo')|(df_seguimiento_ind7['suspendido']=='egresado')]

In [ ]:
df_seguimiento_ind7_clean = df_seguimiento_ind7_clean[df_seguimiento_ind7_clean['cont_estado_seguimiento']!='Aún no contactado']

In [ ]:
df_seguimiento_ind7_clean = df_seguimiento_ind7_clean[df_seguimiento_ind7_clean['cont_inicio_cuarentena'].notna()]

In [ ]:
df_seguimiento_ind7_clean= df_seguimiento_ind7_clean.sort_values(by=['n_folio', 'dia_contacto']).groupby('n_folio').head(1)

In [ ]:
df_seguimiento_ind7_clean['dif_intervalo'] = pd.to_datetime(df_seguimiento_ind7_clean['fecha_cumple_requisitos']) - pd.to_datetime(df_seguimiento_ind7_clean['dia_contacto'])

In [ ]:
df_seguimiento_ind7_clean['dif_intervalo'].value_counts()

In [ ]:
df_seguimiento_ind7_clean['dif_ind7'] = (df_seguimiento_ind7_clean['dif_intervalo'] <= pd.to_timedelta('2 days'))#| (df_seguimiento_ind7_clean['dif_intervalo'] == pd.to_timedelta('-1 days'))|(df_seguimiento_ind7_clean['dif_intervalo'] == pd.to_timedelta('0 days'))

In [ ]:
df_seguimiento_ind7_clean['dif_ind7']

In [ ]:
df_seguimiento_ind7_clean.groupby(['fecha_cumple_requisitos','dif_ind7']).count()[['n_folio']].reset_index()

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_seguimiento_ind7_clean.groupby(['fecha_cumple_requisitos','dif_ind7']).count()[['n_folio']].reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color= 'dif_ind7',
        color_discrete_map={
            True: 'rgb(0,134,149)',
            False: 'rgb(207,28,144)'#rgb(222,203,228)#C075A6
          },
        labels={'dif_ind7':'Investigado en 48h'}, 
       ).update_layout(title= 'Ind 7: Capacidad de investigación de contactos en 48h',
                    yaxis_title='No. de contactos investigados dentro de 48 hrs',
                    xaxis_title='Fecha',
                    template='simple_white',
                      ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0.1, 
                                      #opacity=0.8
                                     )                  
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_seguimiento_ind7_clean.groupby(['fecha_cumple_requisitos','dif_ind7']).count()[['n_folio']].reset_index().merge(
    data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby(['semana_epidemiologica_21','dif_ind7']).mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        color= 'dif_ind7',
        color_discrete_map={
            True: 'rgb(0,134,149)',
            False: 'rgb(207,28,144)'#rgb(222,203,228)#C075A6
          },
        labels={'dif_ind7':'Investigado en 48h'}, 
       ).update_layout(title= 'Ind 6: Capacidad de investigación de contactos estrechos de la Región de Aysén',
                    yaxis_title='No. casos nuevos con contactos estrechos',
                    xaxis_title='Semana epidemiológica',
                    template='simple_white',    ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0.1, 
                                      opacity=0.8
                                     )                     
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
df_ind7= df_seguimiento_ind7_clean.groupby(['fecha_cumple_requisitos', 'dif_ind7']).count()[['n_folio']].reset_index()

In [ ]:
df_ind7_perc = (df_ind7[df_ind7['dif_ind7']==True].set_index('fecha_cumple_requisitos')[['n_folio']]/df_ind7.groupby('fecha_cumple_requisitos').sum('n_folio')[['n_folio']])*100

In [ ]:
px.scatter(df_ind7_perc.reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color='n_folio',
        color_continuous_scale=px.colors.sequential.Mint,
        labels={'n_folio':'Oportunidad (%)'},
        ).update_layout(title= 'Ind 7: Capacidad de investigación de contactos en 48h',
                       yaxis_title='No. de contactos investigados dentro de 48 hrs (%)',
                       xaxis_title='Fecha',
                       template='simple_white',
                 #      xaxis=dict(tickmode = 'array',
                  #                tickvals = list(range(12,88,3)),
                   #               ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                    #             ),
                 #   ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                  #             ).add_hline(y=10, 
                   #                        line_width=3, 
                    #                       line_dash='longdash', 
                     #                      line_color='olive',
                      #                     annotation_text='Meta nacional (10%)',
                       #                    annotation_position='bottom right'
                        #         ).add_hline(y=mean_bac_ind, 
                         #                    line_width=3, 
                          #                   line_dash='longdash', 
                           #                  line_color='green',
                            #                 annotation_text= 'Promedio BAC ('+str(mean_bac_ind)+'%)',
                             #                annotation_position='bottom right'
                              #       ).add_vrect(x0='51', x1='50', 
                               #            annotation_text='2020', 
                                #           annotation_position='top right',
                                 #          annotation=dict(font_size=18, 
                                  #                         font_family="'Open Sans', verdana, arial, sans-serif"),
                                   #                        opacity=0, 
                                    #                       line_width=0
                                     #     ).add_vrect(x0='84', x1='86', 
                                      #     annotation_text='2021', 
                                       #    annotation_position='top right',
                                        #   annotation=dict(font_size=18, 
                                         #                  font_family="'Open Sans', verdana, arial, sans-serif"),
                                          #                 opacity=0, 
                                           #                line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

In [ ]:
mean_ind7 = df_ind7_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean()['n_folio'].mean()

In [ ]:
px.scatter(df_ind7_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        #trendline='rolling',
        #trendline_options=dict(window=5),
        color='n_folio',
        color_continuous_scale=px.colors.sequential.Mint,
        labels={'n_folio':'Oportunidad (%)'},
        ).update_layout(title= 'Ind 7: Capacidad de investigación de contactos en 48h',
                       yaxis_title='Contactos investigados dentro de 48 hrs (%)',
                       xaxis_title='Semana epidemiológica',
                       yaxis_range= [0,105],
                       template='simple_white',
                       xaxis=dict(tickmode = 'array',
                                  tickvals = list(range(12,88,3)),
                                  ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                                 ),
                    ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                              ).add_hline(y=80,
                                            line_width=3, 
                                            line_dash='longdash', 
                                            line_color='#222A2A',
                                            annotation_text='Meta (80%)',
                                            annotation_position='bottom left'
                                 ).add_hline(y=mean_ind7, 
                                             line_width=3, 
                                             line_dash='longdash', 
                                             line_color='#000000',
                                             annotation_text= 'Promedio ('+str(round(mean_ind7, 2))+'%)',
                                             annotation_position='bottom left'
                                     ).add_vrect(x0='51', x1='53', 
                                           annotation_text='2020', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                          ).add_vrect(x0='84', x1='88', 
                                           annotation_text='2021', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

# Indicador 8: Casos en seguimiento

## Objetivo: Evaluar la capacidad de seguimiento de casos

### Meta: >90%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de Información:** base EPIVIGILA actualizada con los reportes de laboratorio al día de la determinación del indicador. Base de seguimiento

### Cálculo del indicador a nivel comunal

$\frac{No.\:casos\:activos\:investigados\:con\:seguimiento\:según\:protocolo\:por\:comuna\:último\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:comuna\:ultimo\:establecimiento\:semana\:x}x100$

### Cálculo del indicador a nivel regional

$\frac{No.\:casos\:activos\:investigados\:con\:seguimiento\:según\:protocolo\:por\:región\:último\:establecimiento\:semana\:x}{No.\:casos\:nuevos\:registrados\:por\:región\:ultimo\:establecimiento\:semana\:x}x100$

**Observación**

- No se incluyen los casos nuevos no notificados ("casos reportados por laboratorio")
- Se considera un caso activo como aquel caso notificado que cumple la definición de caso activo, con fecha de notificación en la semana de interés
- Se consideran sólo registros activos o egresados en la bandeja de seguimiento
- Según protocolo implica un seguimiento diario por día de seguimiento, desde el inicio de seguimiento y se valora de forma dicotómica (cumple o no cumple el protocolo de seguimiento)
- Por seguimiento diario se entiende que diariamente, se debe realizar una actividad de seguimiento (la cual puede ser exitosa o no)

In [ ]:
df_seguimiento_ind8 = df_seguimiento_clean

In [ ]:
# Se consideran sólo registros de casos activos o egresados
df_seguimiento_ind8_clean = df_seguimiento_ind8[(df_seguimiento_ind8['suspendido']=='activo')|(df_seguimiento_ind8['suspendido']=='egresado')]

In [ ]:
#df_seguimiento_ind8_clean = df_seguimiento_ind8_clean[df_seguimiento_ind8_clean['cont_estado_seguimiento']!='Aún no contactado']

In [ ]:
df_seguimiento_ind8_clean = df_seguimiento_ind8_clean[df_seguimiento_ind8_clean['cont_inicio_cuarentena'].notna()]

In [ ]:
df_seguimiento_ind8_clean = df_seguimiento_ind8_clean[df_seguimiento_ind8_clean['tipo_seguimiento']=='caso']

In [ ]:
df_seguimiento_ind8_clean['fecha_cumple_requisitos'] = pd.to_datetime(df_seguimiento_ind8_clean['fecha_cumple_requisitos'])

In [ ]:
# Calcula No. de dias que un caso debiera ser contactado

df_seguimiento_ind8_clean['dif_needed'] =  pd.to_datetime(df_seguimiento_ind8_clean['cont_fin_cuarentena'], errors='coerce')-df_seguimiento_ind8_clean['fecha_cumple_requisitos']

df_seguimiento_ind8_clean['dif_needed'].sample(1)

In [ ]:
df_seguimiento_ind8_clean.sort_values(by=['n_folio', 'dia_contacto'])[['n_folio','fecha_cumple_requisitos','cont_inicio_cuarentena','cont_fin_cuarentena','dia_contacto','fecha_notificacion']].sort_values(by=['n_folio', 'dia_contacto']).groupby('n_folio').head(1)

In [ ]:
# Calcula No. de dias que un caso fue contactado

df_seguimiento_ind8_clean.sort_values(by=['n_folio', 'dia_contacto']).groupby('n_folio').count()[['nfolio_caso']]

In [ ]:
df_seguimiento_ind8_clean.sort_values(by=['n_folio'])[['n_folio','dif_needed']].groupby('n_folio').head(1)

In [ ]:
df_ind8= df_seguimiento_ind8_clean.sort_values(by=['n_folio'])[['n_folio','dif_needed']].groupby('n_folio').head(1).merge(
    df_seguimiento_ind8_clean.sort_values(by=['n_folio', 'dia_contacto']).groupby('n_folio').count()[['nfolio_caso']],
    how='left',
    left_on = 'n_folio',
    right_on = 'n_folio'
)

In [ ]:
df_ind8.sample(3)

In [ ]:
# Transforma datos a formato Float64

df_ind8['nfolio_caso'] = df_ind8['nfolio_caso'].astype('float64')

df_ind8['dif_needed'] = df_ind8['dif_needed'].dt.days

In [ ]:
df_ind8['goal_100'] = df_ind8['nfolio_caso'] >= df_ind8['dif_needed']

df_ind8['goal_80'] = df_ind8['nfolio_caso'] >= df_ind8['dif_needed']*0.8

df_ind8['goal_60'] = df_ind8['nfolio_caso'] >= df_ind8['dif_needed']*0.6

In [ ]:
df_ind8

In [ ]:
# Realiza merge con df original

df_ind8_merged= df_ind8.merge(df_seguimiento_clean,
             how='inner',
             left_on ='n_folio',
             right_on='n_folio'
             ) 

In [ ]:
df_ind8_merged_head = df_ind8_merged.sort_values(['n_folio', 'fecha_notificacion']).groupby('n_folio').head(1)

In [ ]:
df_ind8_merged_head.groupby(['fecha_cumple_requisitos','goal_100']).count()

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_ind8_merged_head.groupby(['fecha_cumple_requisitos','goal_100']).count().reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color= 'goal_100',
        color_discrete_map={
            True: '#620042',
            False: 'rgb(222,203,228)'#rgb(222,203,228)#C075A6
          },
        labels={'goal_100':'Cumplido'}, 
       ).update_layout(title= 'Ind8: Capacidad de seguimiento de casos',
                    yaxis_title='No. casos activos investigados con seguimiento diario',
                    xaxis_title='Fecha',
                    template='simple_white',
                      ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0.1, opacity=0.9)                  
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_ind8_merged_head.groupby(['fecha_cumple_requisitos','goal_100']).count().reset_index().merge(
    data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby(['semana_epidemiologica_21','goal_100']).mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        color= 'goal_100',
        color_discrete_map={
            True: '#620042',
            False: 'rgb(222,203,228)'#rgb(222,203,228)#C075A6
          },
        labels={'goal_100':'Cumplido'}, 
       ).update_layout(title= 'Ind8: Capacidad de seguimiento de casos',
                    yaxis_title='No. casos activos investigados con seguimiento diario',
                    xaxis_title='Semana epidemiológica',
                    template='simple_white',
                      ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0.1, opacity=0.9)                  
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_ind8_merged_head.groupby(['fecha_cumple_requisitos','goal_80']).count().reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color= 'goal_80',
        color_discrete_map={
            True: '#620042',
            False: 'rgb(222,203,228)'#rgb(222,203,228)#C075A6
          },
        labels={'goal_80':'Cumplido'}, 
       ).update_layout(title= 'Ind8: Capacidad de seguimiento de casos',
                    yaxis_title='No. casos activos investigados con seguimiento diario',
                    xaxis_title='Fecha',
                    template='simple_white',
                      ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0.1, opacity=0.9)                 
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_ind8_merged_head.groupby(['fecha_cumple_requisitos','goal_80']).count().reset_index().merge(
    data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby(['semana_epidemiologica_21','goal_80']).mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        color= 'goal_80',
        color_discrete_map={
            True: '#620042',
            False: 'rgb(222,203,228)'#rgb(222,203,228)#C075A6
          },
        labels={'goal_80':'Cumplido'}, 
       ).update_layout(title= 'Ind8: Capacidad de seguimiento de casos',
                    yaxis_title='No. casos activos investigados con seguimiento diario',
                    xaxis_title='Semana epidemiológica',
                    template='simple_white',
                      ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0.1, opacity=0.9)                  
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_ind8_merged_head.groupby(['fecha_cumple_requisitos','goal_60']).count().reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color= 'goal_60',
        color_discrete_map={
            True: '#620042',
            False: 'rgb(222,203,228)'#rgb(222,203,228)#C075A6
          },
        labels={'goal_60':'Cumplido'}, 
       ).update_layout(title= 'Ind8: Capacidad de seguimiento de casos',
                    yaxis_title='No. casos activos investigados con seguimiento diario',
                    xaxis_title='Fecha',
                    template='simple_white',
                      ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0.1, opacity=0.8)                 
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_ind8_merged_head.groupby(['fecha_cumple_requisitos','goal_60']).count().reset_index().merge(
    data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby(['semana_epidemiologica_21','goal_60']).mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        color= 'goal_60',
        color_discrete_map={
            True: '#620042',
            False: 'rgb(222,203,228)'#rgb(222,203,228)#C075A6
          },
        labels={'goal_60':'Cumplido'}, 
       ).update_layout(title= 'Ind8: Capacidad de seguimiento de casos',
                    yaxis_title='No. casos activos investigados con seguimiento diario',
                    xaxis_title='Semana epidemiológica',
                    template='simple_white',
                      ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0.1, opacity=0.9)                  
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
df_ind8_100= df_ind8_merged_head.groupby(['fecha_cumple_requisitos','goal_100']).count()[['n_folio']].reset_index()

In [ ]:
df_ind8_100

In [ ]:
df_ind8_80= df_ind8_merged_head.groupby(['fecha_cumple_requisitos','goal_80']).count()[['n_folio']].reset_index()

In [ ]:
df_ind8_60= df_ind8_merged_head.groupby(['fecha_cumple_requisitos','goal_60']).count()[['n_folio']].reset_index()

In [ ]:
df_ind8_perc100 = (df_ind8_100[df_ind8_100['goal_100']==True].set_index('fecha_cumple_requisitos')[['n_folio']]/df_ind8_100.groupby('fecha_cumple_requisitos').sum('n_folio')[['n_folio']])*100

In [ ]:
df_ind8_perc80 = (df_ind8_80[df_ind8_80['goal_80']==True].set_index('fecha_cumple_requisitos')[['n_folio']]/df_ind8_80.groupby('fecha_cumple_requisitos').sum('n_folio')[['n_folio']])*100

In [ ]:
df_ind8_perc60 = (df_ind8_60[df_ind8_60['goal_60']==True].set_index('fecha_cumple_requisitos')[['n_folio']]/df_ind8_60.groupby('fecha_cumple_requisitos').sum('n_folio')[['n_folio']])*100

In [ ]:
px.scatter(df_ind8_perc100.reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color='n_folio',
        color_continuous_scale=px.colors.sequential.PuRd,
        labels={'n_folio':'Capacidad (%)'},
        ).update_layout(title= 'Ind8: Capacidad de seguimiento de casos',
                       yaxis_title='Casos activos investigados con seguimiento diario (%)',
                       xaxis_title='Fecha',
                       template='simple_white',
                 #      xaxis=dict(tickmode = 'array',
                  #                tickvals = list(range(12,88,3)),
                   #               ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                    #             ),
                 #   ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                  #             ).add_hline(y=10, 
                   #                        line_width=3, 
                    #                       line_dash='longdash', 
                     #                      line_color='olive',
                      #                     annotation_text='Meta nacional (10%)',
                       #                    annotation_position='bottom right'
                        #         ).add_hline(y=mean_bac_ind, 
                         #                    line_width=3, 
                          #                   line_dash='longdash', 
                           #                  line_color='green',
                            #                 annotation_text= 'Promedio BAC ('+str(mean_bac_ind)+'%)',
                             #                annotation_position='bottom right'
                              #       ).add_vrect(x0='51', x1='50', 
                               #            annotation_text='2020', 
                                #           annotation_position='top right',
                                 #          annotation=dict(font_size=18, 
                                  #                         font_family="'Open Sans', verdana, arial, sans-serif"),
                                   #                        opacity=0, 
                                    #                       line_width=0
                                     #     ).add_vrect(x0='84', x1='86', 
                                      #     annotation_text='2021', 
                                       #    annotation_position='top right',
                                        #   annotation=dict(font_size=18, 
                                         #                  font_family="'Open Sans', verdana, arial, sans-serif"),
                                          #                 opacity=0, 
                                           #                line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

In [ ]:
mean_ind8_perc100 = df_ind8_perc100.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean()['n_folio'].mean()

In [ ]:
mean_ind8_perc80 = df_ind8_perc80.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean()['n_folio'].mean()

In [ ]:
mean_ind8_perc60 = df_ind8_perc60.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean()['n_folio'].mean()

In [ ]:
px.scatter(df_ind8_perc100.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        #trendline='rolling',
        #trendline_options=dict(window=5),
        color='n_folio',
        color_continuous_scale=px.colors.sequential.PuRd,
        labels={'n_folio':'Capacidad (%)'},
        ).update_layout(title= 'Ind8: Capacidad de seguimiento de casos',
                       yaxis_title='No. casos activos investigados con seguimiento diario (%)',
                       xaxis_title='Semana epidemiológica',
                       yaxis_range= [0,105],
                       template='simple_white',
                       xaxis=dict(tickmode = 'array',
                                  tickvals = list(range(12,88,3)),
                                  ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                                 ),
                    ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                              ).add_hline(y=80,
                                            line_width=3, 
                                            line_dash='longdash', 
                                            line_color='#222A2A',
                                            annotation_text='Meta (90%)',
                                            annotation_position='bottom left'
                                 ).add_hline(y=mean_ind8_perc100, 
                                             line_width=3, 
                                             line_dash='longdash', 
                                             line_color='#750D86',
                                             annotation_text= 'Promedio ('+str(round(mean_ind8_perc100, 2))+'%)',
                                             annotation_position='bottom left'
                                     ).add_vrect(x0='51', x1='53', 
                                           annotation_text='2020', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                          ).add_vrect(x0='84', x1='88', 
                                           annotation_text='2021', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

In [ ]:
px.scatter(df_ind8_perc80.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        #trendline='rolling',
        #trendline_options=dict(window=5),
        color='n_folio',
        color_continuous_scale=px.colors.sequential.PuRd,
        labels={'n_folio':'Capacidad (%)'},
        ).update_layout(title= 'Ind8: Capacidad de seguimiento de casos',
                       yaxis_title='No. casos activos investigados con seguimiento diario (%)',
                       xaxis_title='Semana epidemiológica',
                       yaxis_range= [0,105],
                       template='simple_white',
                       xaxis=dict(tickmode = 'array',
                                  tickvals = list(range(12,88,3)),
                                  ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                                 ),
                    ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                              ).add_hline(y=80,
                                            line_width=3, 
                                            line_dash='longdash', 
                                            line_color='#222A2A',
                                            annotation_text='Meta (90%)',
                                            annotation_position='bottom left'
                                 ).add_hline(y=mean_ind8_perc80, 
                                             line_width=3, 
                                             line_dash='longdash', 
                                             line_color='#750D86',
                                             annotation_text= 'Promedio ('+str(round(mean_ind8_perc80, 2))+'%)',
                                             annotation_position='bottom left'
                                     ).add_vrect(x0='51', x1='53', 
                                           annotation_text='2020', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                          ).add_vrect(x0='84', x1='88', 
                                           annotation_text='2021', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

In [ ]:
px.scatter(df_ind8_perc60.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        #trendline='rolling',
        #trendline_options=dict(window=5),
        color='n_folio',
        color_continuous_scale=px.colors.sequential.PuRd,
        labels={'n_folio':'Capacidad (%)'},
        ).update_layout(title= 'Ind8: Capacidad de seguimiento de casos',
                       yaxis_title='No. casos activos investigados con seguimiento diario (%)',
                       xaxis_title='Semana epidemiológica',
                       yaxis_range= [0,105],
                       template='simple_white',
                       xaxis=dict(tickmode = 'array',
                                  tickvals = list(range(12,88,3)),
                                  ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                                 ),
                    ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                              ).add_hline(y=80,
                                            line_width=3, 
                                            line_dash='longdash', 
                                            line_color='#222A2A',
                                            annotation_text='Meta (90%)',
                                            annotation_position='top left'
                                 ).add_hline(y=mean_ind8_perc60, 
                                             line_width=3, 
                                             line_dash='longdash', 
                                             line_color='#750D86',
                                             annotation_text= 'Promedio ('+str(round(mean_ind8_perc60, 2))+'%)',
                                             annotation_position='bottom left'
                                     ).add_vrect(x0='51', x1='53', 
                                           annotation_text='2020', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                          ).add_vrect(x0='84', x1='88', 
                                           annotation_text='2021', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

# Indicador 10: Contactos con al menos 2 seguimientos dentro de su periodo de cuarentena de 14 días

## Objetivo: Evaluar la capacidad de seguimiento de contactos estrechos

### Meta: >90%

**Periodicidad:** Semanal

**Escala territorial:** Comuna, Regional, Nacional

**Fuente de información:** Base de seguimiento

### Cálculo del indicador a nivel comunal

$\frac{No.\:contactos\:que\:cumplieron\:cuarentena\:con\:al\:menos\:2\:seguimientos\:por\:comuna\:último\:establecimiento\:semana\:x}{No.\:contactos\:que\:cumplieron\:cuarentena\:por\:comuna\:ultimo\:establecimiento\:semana\:x}x100$

### Cálculo del indicador a nivel regional

$\frac{No.\:contactos\:que\:cumplieron\:cuarentena\:con\:al\:menos\:2\:seguimientos\:por\:región\:último\:establecimiento\:semana\:x}{No.\:contactos\:que\:cumplieron\:cuarentena\:por\:región\:ultimo\:establecimiento\:semana\:x}x100$

**Observación**

- El indicador se construye sobre los contactos que egresan (fecha de término de cuarentena) de seguimiento, durante la semana de interés (esto es, sólo registros egresados de la bandeja de seguimiento)
- El periodo de seguimiento para el contacto corresponde a la cantidad de días entre la 1era fecha en que el contacto está disponible en la bandeja de seguimiento hasta la última fecha fin cuarentena
- Se cumple con el indicador si en el período de seguimiento del contacto se registran al menos 2 actividades de seguimiento (las cuales pueden ser exitosas o no)
- Se excluyen los contactos que se transforman en caso y que están notificados (tiene folio asociado como caso)

In [ ]:
df_seguimiento_ind10 = df_seguimiento_clean

In [ ]:
# Se consideran sólo registros de casos activos o egresados
df_seguimiento_ind10_clean = df_seguimiento_ind10[(df_seguimiento_ind10['suspendido']=='egresado')]

In [ ]:
# No se convierte en caso 

df_seguimiento_ind10_clean = df_seguimiento_ind10_clean[df_seguimiento_ind10_clean['folio_se_convierte_en_caso']!='t']

In [ ]:
df_seguimiento_ind10_clean

In [ ]:
# Se incluyen solo los tipos de seguimiento contacto

df_seguimiento_ind10_clean = df_seguimiento_ind10_clean[df_seguimiento_ind10_clean['tipo_seguimiento']=='contacto']

In [ ]:
# Se excluyen datos de laboratorio

df_seguimiento_ind10_clean = df_seguimiento_ind10_clean[df_seguimiento_ind10_clean['nfolio_pntm'].isnull()]

In [ ]:
df_seguimiento_ind10_clean

In [ ]:
# Calcula veces que un contacto fue contactado para seguimiento

ind10_contactado= df_seguimiento_ind10_clean.sort_values(by=['n_folio', 'dia_contacto']).groupby('n_folio').count()[['nfolio_contacto_estrecho']].rename(columns={'nfolio_contacto_estrecho':'veces_contactado'})

ind10_contactado.sample(1)

In [ ]:
df_ind10_contactado = df_seguimiento_ind10_clean.merge(
                            ind10_contactado,
                            how='left',
                            left_on='n_folio',
                            right_on=ind10_contactado.index

                        )

In [ ]:
df_ind10_contactado.sample(1)

In [ ]:
# Crea variable ind10_dos y ind10_tres representando metas de más de dos y tres contactos respectivamente

df_ind10_contactado['ind10_dos'] = df_ind10_contactado['veces_contactado']>=2
df_ind10_contactado['ind10_tres'] = df_ind10_contactado['veces_contactado']>=3

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_ind10_contactado.groupby(['fecha_cumple_requisitos','ind10_dos']).count()[['n_folio']].reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color= 'ind10_dos',
               color_discrete_map={
                     True: '#3366CC',
                     False: '#FF6692'
                     },
        labels={'ind10_dos':'2 o más seguimientos'}, 
       ).update_layout(title= 'Ind10: Capacidad de seguimiento de contactos estrechos (2 o más seguimientos)',
                    yaxis_title='No. contactos estrechos con cuarentena',
                    xaxis_title='Fecha',
                    template='simple_white',
                      ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0.1, 
                                      #opacity=0.8
                                     )                 
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_ind10_contactado.groupby(['fecha_cumple_requisitos','ind10_dos']).count()[['n_folio']].reset_index().merge(
    data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby(['semana_epidemiologica_21','ind10_dos']).mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        color= 'ind10_dos',
               color_discrete_map={
                     True: '#3366CC',
                     False: '#FF6692'
                     },
        labels={'ind10_dos':'2 o más seguimientos'}, 
       ).update_layout(title= 'Ind10: Capacidad de seguimiento de contactos estrechos (2 o más seguimientos)',
                    yaxis_title='No. contactos estrechos con cuarentena',
                    xaxis_title='Semana epidemiológica',
                    template='simple_white',
                      ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0.1, 
                                      #opacity=0.8
                                     )                 
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_ind10_contactado.groupby(['fecha_cumple_requisitos','ind10_tres']).count()[['n_folio']].reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'n_folio',
        color= 'ind10_tres',
        color_discrete_map={
                     True: '#3366CC',
                     False: '#FF6692'
                     },
        labels={'ind10_tres':'3 o más seguimientos'}, 
       ).update_layout(title= 'Ind10: Capacidad de seguimiento de contactos estrechos (3 o más seguimientos)',
                    yaxis_title='No. contactos estrechos con cuarentena',
                    xaxis_title='Fecha',
                    template='simple_white', 
                      ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0.1, 
                                      #opacity=0.8
                                     )                  
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
# Crea gráfico de etapa_clinica regional por semana epidemiologica

px.bar(df_ind10_contactado.groupby(['fecha_cumple_requisitos','ind10_tres']).count()[['n_folio']].reset_index().merge(
    data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby(['semana_epidemiologica_21','ind10_tres']).mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'n_folio',
        color= 'ind10_tres',
               color_discrete_map={
                     True: '#3366CC',
                     False: '#FF6692'
                     },
        labels={'ind10_tres':'3 o más seguimientos'}, 
       ).update_layout(title= 'Ind10: Capacidad de seguimiento de contactos estrechos (3 o más seguimientos)',
                    yaxis_title='No. contactos estrechos con cuarentena',
                    xaxis_title='Semana epidemiológica',
                    template='simple_white',
                      ).update_traces(marker_line_color='rgb(0,0,0)',
                                      marker_line_width=0.1, 
                                      #opacity=0.8
                                     )                 
                    #xaxis=dict(tickmode = 'array',
                     #             tickvals = list(range(12,88,3)),
                      #            ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                      #           ),
               #     ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                #               ).add_vrect(x0='50', x1='50', 
                 #                          annotation_text='2020', 
                  #                         annotation_position='top right',
                   #                        annotation=dict(font_size=18, 
                    #                                       font_family="'Open Sans', verdana, arial, sans-serif"),
                     #                                      opacity=0, 
                      #                                     line_width=0
                       #                   ).add_vrect(x0='84', x1='86', 
                        #                   annotation_text='2021', 
                         #                  annotation_position='top right',
                          #                 annotation=dict(font_size=18, 
                           #                                font_family="'Open Sans', verdana, arial, sans-serif"),
                            #                               opacity=0, 
                             #                              line_width=0)  
                      

In [ ]:
df_ind10_tres= df_ind10_contactado.groupby(['fecha_cumple_requisitos', 'ind10_tres']).count()[['n_folio']].reset_index()

In [ ]:
df_ind10_tres.tail(3)

In [ ]:
df_ind10_dos= df_ind10_contactado.groupby(['fecha_cumple_requisitos', 'ind10_dos']).count()[['n_folio']].reset_index()

In [ ]:
df_ind10_dos.tail(3)

In [ ]:
df_ind10_perc = (df_ind10_tres[df_ind10_tres['ind10_tres']==True].set_index('fecha_cumple_requisitos')[['n_folio']]/df_ind10_tres.groupby('fecha_cumple_requisitos').sum('n_folio')[['n_folio']])*100

In [ ]:
df_ind10_perc['ind10_dos_perc'] = (df_ind10_dos[df_ind10_dos['ind10_dos']==True].set_index('fecha_cumple_requisitos')[['n_folio']]/df_ind10_dos.groupby('fecha_cumple_requisitos').sum('n_folio')[['n_folio']])*100

In [ ]:
df_ind10_perc= df_ind10_perc.rename(columns={'n_folio':'ind10_tres_perc'})


In [ ]:
px.scatter(df_ind10_perc.reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'ind10_dos_perc',
        color='ind10_dos_perc',
        color_continuous_scale=px.colors.sequential.Plotly3_r,
        labels={'ind10_dos_perc':'Oportunidad (%)'},
        ).update_layout(title= 'Ind10: Capacidad de seguimiento de contactos estrechos (2 o más seguimientos)',
                       yaxis_title='Contactos estrechos con 2 o más seguimientos (%)',
                       xaxis_title='Fecha',
                       template='simple_white',
                 #      xaxis=dict(tickmode = 'array',
                  #                tickvals = list(range(12,88,3)),
                   #               ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                    #             ),
                 #   ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                  #             ).add_hline(y=10, 
                   #                        line_width=3, 
                    #                       line_dash='longdash', 
                     #                      line_color='olive',
                      #                     annotation_text='Meta nacional (10%)',
                       #                    annotation_position='bottom right'
                        #         ).add_hline(y=mean_bac_ind, 
                         #                    line_width=3, 
                          #                   line_dash='longdash', 
                           #                  line_color='green',
                            #                 annotation_text= 'Promedio BAC ('+str(mean_bac_ind)+'%)',
                             #                annotation_position='bottom right'
                              #       ).add_vrect(x0='51', x1='50', 
                               #            annotation_text='2020', 
                                #           annotation_position='top right',
                                 #          annotation=dict(font_size=18, 
                                  #                         font_family="'Open Sans', verdana, arial, sans-serif"),
                                   #                        opacity=0, 
                                    #                       line_width=0
                                     #     ).add_vrect(x0='84', x1='86', 
                                      #     annotation_text='2021', 
                                       #    annotation_position='top right',
                                        #   annotation=dict(font_size=18, 
                                         #                  font_family="'Open Sans', verdana, arial, sans-serif"),
                                          #                 opacity=0, 
                                           #                line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

In [ ]:
px.scatter(df_ind10_perc.reset_index(),
        x= 'fecha_cumple_requisitos',
        y= 'ind10_tres_perc',
        color='ind10_tres_perc',
        color_continuous_scale=px.colors.sequential.Plotly3_r,
        labels={'ind10_tres_perc':'Oportunidad (%)'},
        ).update_layout(title= 'Ind10: Capacidad de seguimiento de contactos estrechos (3 o más seguimientos)',
                       yaxis_title='Contactos estrechos con 3 o más seguimientos (%)',
                       xaxis_title='Fecha',
                       template='simple_white',
                 #      xaxis=dict(tickmode = 'array',
                  #                tickvals = list(range(12,88,3)),
                   #               ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                    #             ),
                 #   ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                  #             ).add_hline(y=10, 
                   #                        line_width=3, 
                    #                       line_dash='longdash', 
                     #                      line_color='olive',
                      #                     annotation_text='Meta nacional (10%)',
                       #                    annotation_position='bottom right'
                        #         ).add_hline(y=mean_bac_ind, 
                         #                    line_width=3, 
                          #                   line_dash='longdash', 
                           #                  line_color='green',
                            #                 annotation_text= 'Promedio BAC ('+str(mean_bac_ind)+'%)',
                             #                annotation_position='bottom right'
                              #       ).add_vrect(x0='51', x1='50', 
                               #            annotation_text='2020', 
                                #           annotation_position='top right',
                                 #          annotation=dict(font_size=18, 
                                  #                         font_family="'Open Sans', verdana, arial, sans-serif"),
                                   #                        opacity=0, 
                                    #                       line_width=0
                                     #     ).add_vrect(x0='84', x1='86', 
                                      #     annotation_text='2021', 
                                       #    annotation_position='top right',
                                        #   annotation=dict(font_size=18, 
                                         #                  font_family="'Open Sans', verdana, arial, sans-serif"),
                                          #                 opacity=0, 
                                           #                line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

In [ ]:
px.scatter(df_ind10_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'ind10_tres_perc',
        #trendline='rolling',
        #trendline_options=dict(window=5),
        color='ind10_tres_perc',
        color_continuous_scale=px.colors.sequential.Plotly3_r,
        labels={'ind10_tres_perc':'Oportunidad (%)'},
        ).update_layout(title= 'Ind10: Capacidad de seguimiento de contactos estrechos (3 o más seguimientos)',
                       yaxis_title='Contactos estrechos con 3 o más seguimientos (%)',
                       xaxis_title='Semana epidemiológica',
                       #yaxis_range= [0,105],
                       template='simple_white',
                       xaxis=dict(tickmode = 'array',
                                  tickvals = list(range(12,88,3)),
                                  ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                                 ),
                    ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                              ).add_hline(y=90,
                                            line_width=3, 
                                            line_dash='longdash', 
                                            line_color='#222A2A',
                                            annotation_text='Meta (90%)',
                                            annotation_position='bottom left'
                                # ).add_hline(y=mean_ind7, 
                                 #            line_width=3, 
                                  #           line_dash='longdash', 
                                   #          line_color='#750D86',
                                    #         annotation_text= 'Promedio ('+str(round(mean_ind7, 2))+'%)',
                                     #        annotation_position='bottom left'
                                     ).add_vrect(x0='51', x1='53', 
                                           annotation_text='2020', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                          ).add_vrect(x0='84', x1='88', 
                                           annotation_text='2021', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)

In [ ]:
px.scatter(df_ind10_perc.reset_index().merge(data_aysen_clean_21[['fecha_notificacion','semana_epidemiologica_21']], 
                   left_on='fecha_cumple_requisitos',
                   right_on='fecha_notificacion',
                   how='left').groupby('semana_epidemiologica_21').mean().reset_index(),
        x= 'semana_epidemiologica_21',
        y= 'ind10_dos_perc',
        #trendline='rolling',
        #trendline_options=dict(window=5),
        color='ind10_dos_perc',
        color_continuous_scale=px.colors.sequential.Plotly3_r,
        labels={'ind10_dos_perc':'Oportunidad (%)'},
        ).update_layout(title= 'Ind10: Capacidad de seguimiento de contactos estrechos (2 o más seguimientos)',
                       yaxis_title='Contactos estrechos con 2 o más seguimientos (%)',
                       xaxis_title='Semana epidemiológica',
                      # yaxis_range= [0,105],
                       template='simple_white',
                       xaxis=dict(tickmode = 'array',
                                  tickvals = list(range(12,88,3)),
                                  ticktext = list(list(range(12,54,3))+list(range(1,88,3))),
                                 ),
                    ).add_vline(x=53.5, line_width=3, line_dash='dash', line_color='gray'
                              ).add_hline(y=90,
                                            line_width=3, 
                                            line_dash='longdash', 
                                            line_color='#222A2A',
                                            annotation_text='Meta (90%)',
                                            annotation_position='bottom left'
                                # ).add_hline(y=mean_ind7, 
                                 #            line_width=3, 
                                  #           line_dash='longdash', 
                                   #          line_color='#750D86',
                                    #         annotation_text= 'Promedio ('+str(round(mean_ind7, 2))+'%)',
                                     #        annotation_position='bottom left'
                                     ).add_vrect(x0='51', x1='53', 
                                           annotation_text='2020', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                          ).add_vrect(x0='84', x1='88', 
                                           annotation_text='2021', 
                                           annotation_position='bottom right',
                                           annotation=dict(font_size=18, 
                                                           font_family="'Open Sans', verdana, arial, sans-serif"),
                                                           opacity=0, 
                                                           line_width=0
                                                     ).update_traces(mode='lines+markers', 
                                                                     line_color='grey',
                                                                     marker_line_width=1, 
                                                                     marker_size=10)